# Ejercicio 1

La proporción de oyentes se compone por un 90% de viejos y 10% de jovenes.

Tabla de probabilidades condicionadas ( P(Columna | Fila) ):

| --- | Programa 1 | Programa 2 | Programa 3 | Programa 4 |
| --- | --- | --- | --- | --- |
| Joven | 0.95 | 0.05 | 0.02 | 0.2 |
| Viejo | 0.03 | 0.82 | 0.34 | 0.92 |

Representamos los eventos de la siguiente forma:
- J: el oyente es joven
- V: el oyente es viejo
- Gi: al oyente le gusta el programa i
***
Queremos encontrar la probabilidad de que sea joven dado que le gustan los programas 1 y 3, y no le gustan los programas 2 y 4. Es decir:
<br><br>
P(J | G1, G3, !G2, !G4)
<br><br>
Por el teorema de Bayes obtenemos que:
<br><br>
P(J | G1, G3, !G2, !G4) = P(G1, G3, !G2, !G4 | J).P(J) / P(G1, G3, !G2, !G4)
<br><br>
Y si asumimos independencia condicionada entre sucesos y aplicando el teorema de probabilidad total en el denominador:
<br><br>
P(J | G1, G3, !G2, !G4) = P(G1|J).P(G3|J).P(!G2|J).P(!G4|J).P(J) / (P(G1, G3, !G2, !G4 | J).P(J) + P(G1, G3, !G2, !G4 | V).P(V))
<br><br>
Finalmente, utilizando los datos del enunciado:
<br><br>
P(J | G1, G3, !G2, !G4) = 0.92
***
Ahora queremos encontrar la probabilidad de que el oyente sea viejo dado los mismos gustos. Es decir:
<br><br>
P(V | G1, G3, !G2, !G4)
<br><br>
Considerando que el oyente o es viejo o es joven:
<br><br>
P(V | G1, G3, !G2, !G4) = P(!J | G1, G3, !G2, !G4) = 1 - P(J | G1, G3, !G2, !G4) = 0.08

# Ejercicio 2

Importamos las librerias

In [ ]:
import pandas as pd
from TP1.src.Ej2 import divide_by_nationality
from TP1.src.NaiveBayes import NaiveBayes

Leemos los datos y los dividimos por nacionalidad

In [ ]:
data = pd.read_excel('../PreferenciasBritanicos.xlsx', sheet_name='Hoja1', )

britanicos = divide_by_nationality(data, 'I')
escocesas = divide_by_nationality(data, 'E')

datasets = [britanicos, escocesas]

Entrenamos Bayes

In [ ]:
nb = NaiveBayes()
nb.train(datasets)

Buscamos la nacionalidad con las caracteristicas solicitadas

In [ ]:
category = nb.calculate_category([1, 0, 1, 1, 0])

print("Es de Nacionalidad :" + str(datasets[category]['Nacionalidad'].values[0]))

# Ejercicio 3

Importamos el dataset y librerias.

In [2]:
from TP1.src.Ej3 import add_columns_from_text, get_categories, separate_in_categories, confusion_matrix, get_metrics, \
    invert_dict, roc_curve
import pandas as pd
from TP1.src.NaiveBayes import NaiveBayes

df = pd.read_csv('../dataset/Noticias_argentinas.csv', nrows=1000)

df = df[['titular', 'categoria']]

A partir de los titulares generamos una matriz donde sus columnas son las palabras que se utilizaron
 para escribir todos los titulares del dataset. Es decir la union de los conjuntos de palabras utilizados para cada
 titular.

 Las filas de la matriz serán cada una de las noticias (igual que antes) y el valor que tomarán para cada
 columna será 1 si la palabra de la columna aparece en el titular de esta noticia y 0 en el caso de que no aparezca.

In [3]:

df = add_columns_from_text(df)
print(df)

                                               titular categoria  \
0    Trabajadores del Buenos Aires Design cortan la...  Nacional   
1    La boda del gobernador Gerardo Morales: tapas ...  Nacional   
2    Cumbre del G20: qué calles estarán cortadas y ...  Nacional   
3    Una fractura que confirma la candidatura de Cr...  Nacional   
4    Infierno grande: ola de divorcios en un pueblo...  Nacional   
..                                                 ...       ...   
995  Que de la mano de Maradona, todos la vuelta va...  Deportes   
996         El Halcón y el Pincha completan su partido  Deportes   
997  El enojo de Lionel Scaloni con un periodista e...  Deportes   
998          Los entrenadores de River con más títulos  Deportes   
999  El fuerte mensaje de Migue Granados para fomen...     Salud   

     trabajadores  del  buenos  aires  design  cortan  la  avenida  ...  \
0               1    1       1      1       1       1   1        1  ...   
1               0    1       0   

Obtenemos las 4 categorias que vamos a clasificar.

In [ ]:
categories = get_categories(df)[0:4]

Separamos el dataset en training y test data.

In [ ]:
train = df.sample(frac=0.8)
test = df.drop(train.index)

Particionamos los dataframes en categorias y luego eliminamos las columnas innecesarias.

In [ ]:
train_datasets = separate_in_categories(train, categories)
test_datasets = separate_in_categories(test, categories)

train_datasets = [df.drop(columns=['titular', 'categoria']) for df in train_datasets]
test_datasets = [df.drop(columns=['titular', 'categoria']) for df in test_datasets]

Creamos una instancia de nuestra implementación de Naive Bayes y entrenamos con nuestro dataset de entrenamiento.

In [ ]:
nb = NaiveBayes()
nb.train(train_datasets)

Generamos la matriz de confusión utilizando el dataset de prueba.

In [ ]:
res = confusion_matrix(nb, test_datasets)
res_dic = {categories[i]: {categories[j]: res[i][j] for j in range(len(res[i]))} for i in range(len(res))}

print(pd.DataFrame.from_dict(res_dic))

Calculamos métricas de evaluación.

In [ ]:
metrics_dic = get_metrics(res, categories)
print(pd.DataFrame.from_dict(invert_dict(metrics_dic)))

Generamos un espacio ROC para graficar los resultados para cada categoria.
Cada punto roc representa la cantidad de TP y FP sobre una categoria particular.

In [ ]:
roc_curve(metrics_dic)

# Ejercicio 4

Importamos las librerias

In [ ]:
from TP1.src.BayesNetwork import BayesNetwork
import pandas as pd

Leemos los datos y los discretizamos

In [ ]:
data = pd.read_csv('../dataset/binary.csv')

data['gre'] = (data['gre'] >= 500).astype(int)
data['gpa'] = (data['gpa'] >= 3).astype(int)

Creamos la red y calculamos las probabilidades

In [ ]:
bn = BayesNetwork(data, {'rank': [], 'gre': ['rank'], 'gpa': ['rank'], 'admit': ['gre', 'gpa', 'rank']}, {'rank': [1, 2, 3, 4], 'gre': [0, 1], 'gpa': [0, 1], 'admit': [0, 1]})

print("P(admit = 0 | rank = 1) = " + str(bn.conditional_probability({'admit': 0}, {'rank': 1})))
print("P(admit = 1 | rank = 2, gre = 0, gpa = 1) = " + str(bn.conditional_probability({'admit': 1}, {'rank': 2, 'gre': 0, 'gpa': 1})))

Leemos los datos y los discretizamos

In [ ]:
data = pd.read_csv('../dataset/binary.csv')

data['gre'] = (data['gre'] >= 500).astype(int)
data['gpa'] = (data['gpa'] >= 3).astype(int)

Creamos la red y calculamos las probabilidades

In [ ]:
bn = BayesNetwork(data, {'rank': [], 'gre': ['rank'], 'gpa': ['rank'], 'admit': ['gre', 'gpa', 'rank']}, {'rank': [1, 2, 3, 4], 'gre': [0, 1], 'gpa': [0, 1], 'admit': [0, 1]})

print("P(admit = 0 | rank = 1) = " + str(bn.conditional_probability({'admit': 0}, {'rank': 1})))
print("P(admit = 1 | rank = 2, gre = 0, gpa = 1) = " + str(bn.conditional_probability({'admit': 1}, {'rank': 2, 'gre': 0, 'gpa': 1})))